In [1]:
print("test")
import os
from pathlib import Path
import sys
node_type = os.getenv('BB_CPU')
venv_dir = f'/rds/homes/g/gaddcz/Projects/CPRD/virtual-env-{node_type}'
venv_site_pkgs = Path(venv_dir) / 'lib' / f'python{sys.version_info.major}.{sys.version_info.minor}' / 'site-packages'
if venv_site_pkgs.exists():
    sys.path.insert(0, str(venv_site_pkgs))
    print(f"Added path '{venv_site_pkgs}' at start of search paths.")
else:
    print(f"Path '{venv_site_pkgs}' not found. Check that it exists and/or that it exists for node-type '{node_type}'.")

%load_ext autoreload
%autoreload 2

%env SQLITE_TMPDIR=/rds/projects/g/gokhalkm-optimal/DataforCharles
%env TMPDIR=/rds/projects/g/gokhalkm-optimal/DataforCharles
!echo $SQLITE_TMPDIR
!echo $TMPDIR
!echo $USERPROFILE

test
Added path '/rds/homes/g/gaddcz/Projects/CPRD/virtual-env-icelake/lib/python3.10/site-packages' at start of search paths.
env: SQLITE_TMPDIR=/rds/projects/g/gokhalkm-optimal/DataforCharles
env: TMPDIR=/rds/projects/g/gokhalkm-optimal/DataforCharles
/rds/projects/g/gokhalkm-optimal/DataforCharles
/rds/projects/g/gokhalkm-optimal/DataforCharles



In [2]:
import pytorch_lightning
import torch
import numpy as np
import matplotlib.pyplot as plt
import random
import sqlite3
import logging
import time
from CPRD.data.database.build_static_db import Static
from CPRD.data.database.build_diagnosis_db import Diagnoses
from CPRD.data.database.build_measurements_and_tests_db import Measurements

torch.manual_seed(1337)
logging.basicConfig(level=logging.DEBUG)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"Using device: {device}.")

# !export TMPDIR=/rds/projects/g/gokhalkm-optimal/DataforCharles/tmp
!pwd


Using device: cuda.
/rds/homes/g/gaddcz/Projects/CPRD/data/database


In [3]:
# import pandas as pd
# generator = pd.read_csv(PATH_TO_STATIC, chunksize=10, iterator=True, encoding='utf-8', low_memory=False,
#                                dtype={'PATIENT_ID': 'str'}
#                                )
# for _idx, df in enumerate(generator):
#     display(df["IMD"])
#     if _idx > 100:
#         break

In [11]:
PATH_TO_DB = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/cprd.db"

## Create/load tables

In [12]:
PATH_TO_STATIC = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/baseline/masterDataOptimalWithIMD_v3.csv"
PATH_TO_DIAGNOSIS = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/baseline/masterDataOptimalWithIMD_v3.csv"
PATH_TO_MEASUREMENTS = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/timeseries/measurement_and_tests/lab_measurements/"
PATH_TO_MEDICATIONS = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/timeseries/medications/"

load = True

static = Static(PATH_TO_DB, PATH_TO_STATIC, load=load)
diagnosis = Diagnoses(PATH_TO_DB, PATH_TO_DIAGNOSIS, load=load)
measurements = Measurements(PATH_TO_DB, PATH_TO_MEASUREMENTS, load=load)
medications = Measurements(PATH_TO_DB, PATH_TO_MEDICATIONS, load=load)

## View tables

These are built through a job script

In [13]:
for table in [static, diagnosis, measurements, medications]:
    print(table)

DEBUG:root:Connected to SQLite database
DEBUG:root:Disconnected from SQLite database
DEBUG:root:Connected to SQLite database


Static table with 28.12M records.


DEBUG:root:Disconnected from SQLite database
DEBUG:root:Connected to SQLite database


Diagnosis table with 51.00M records.


KeyboardInterrupt: 

## Temporary manual fix to rename table names (so i dont need to rebuild the entire database from a bug fix)

In [26]:
# for table in measurements.measurement_table_names:
#     # print(table)
#     prefix = "measurement_AVF1_masterDataOptimal_v3_fullDB20231112044822_"
#     if table.startswith(prefix):
#         new_table = "measurement_" + table[len(prefix):]
#         print(table)
#         print(new_table)
#         measurements.cursor.execute(f"ALTER TABLE {table}  RENAME TO {new_table};")

In [25]:
measurements.connect()

print(measurements.measurement_table_names)

DEBUG:root:Connected to SQLite database


['measurement_25_Hydroxyvitamin_D2_level_92', 'measurement_25_Hydroxyvitamin_D3_level_90', 'measurement_AST___aspartate_transam_SGOT__46', 'measurement_AST_serum_level_47', 'measurement_Albumin___creatinine_ratio_37', 'measurement_Basophil_count_22', 'measurement_Blood_calcium_level_38', 'measurement_Blood_urea_28', 'measurement_Body_mass_index_3', 'measurement_Brain_natriuretic_peptide_level_66', 'measurement_Calcium_adjusted_level_41', 'measurement_Calculated_LDL_cholesterol_level_103', 'measurement_Combined_total_vitamin_D2_and_D3_level_93', 'measurement_Corrected_serum_calcium_level_42', 'measurement_Current_smoker_83', 'measurement_Diastolic_blood_pressure_5', 'measurement_Eosinophil_count_21', 'measurement_Erythrocyte_sedimentation_rate_61', 'measurement_Ex_smoker_84', 'measurement_Free_T4_level_76', 'measurement_GFR_calculated_abbreviated_MDRD_34', 'measurement_Haematocrit___PCV_16', 'measurement_Haematocrit_15', 'measurement_Haemoglobin_A1c_level___IFCC_standardised_6', 'measur

In [8]:

# measurements.cursor.execute(""" DROP TABLE measurement_SmokingStatus_NeverSmoked;""")
diagnosis.connect()
# diagnosis.cursor.execute(""" ALTER TABLE diagnosis_table
#                              RENAME COLUMN PRACTICE_PATIENT_ID TO PATIENT_ID;""")



DEBUG:root:Connected to SQLite database


## Test speed of SQLite table read

In [ ]:

for mtable in measurements.measurement_table_names:
    measurements.cursor.execute(f"""SELECT
                                    	COUNT(*)
                                    FROM
                                    	{mtable}
                                    """)
    # WHERE PRACTICE_PATIENT_ID = 'p20684_2602314020684' 
    a = measurements.cursor.fetchall()[0][0]
    print(mtable.ljust(50) + f"{a}")
    count += a
# print(a[:10])
print(count)

In [28]:
import time

measurements.connect()

t = time.time()   #  EXPLAIN QUERY PLAN        diagnosis_table           measurement_table        static_table
measurements.cursor.execute(""" SELECT EVENT, COUNT(EVENT)
                                FROM
                                	diagnosis_table
                                 GROUP BY 
                                     EVENT
                                 ;""")
total_count = 0
for condition, count in measurements.cursor.fetchall():
    print(f"{condition}".replace("_"," ").ljust(40) + f" & ${count:,}$ \\\\".rjust(10))
    total_count += count
print("Total".ljust(40) + f" & {total_count:,}")

eval_time = (time.time() - t)
print(eval_time )

ADDISONS DISEASE                         & $6,691$ \\
ADDISON DISEASE                          & $11,794$ \\
AF                                       & $731,332$ \\
ALCOHOLMISUSE V2                         & $1,125,212$ \\
ALLCANCER NOHAEM NOBCC                   & $1,496,973$ \\
ALLERGICRHINITISCONJ                     & $3,291,165$ \\
ALL DEMENTIA                             & $528,602$ \\
ANXIETY                                  & $3,560,978$ \\
ANY DEAFNESS HEARING LOSS V2             & $2,282,766$ \\
AORTICANEURYSM V2                        & $101,134$ \\
ASTHMA PUSHASTHMA                        & $4,175,115$ \\
ATOPICECZEMA                             & $4,369,082$ \\
AUTISM                                   & $156,860$ \\
BIPOLAR                                  & $108,852$ \\
BRONCHIECTASIS                           & $112,618$ \\
CHRONICFATIGUESYNDROMEMM V2              & $82,799$ \\
CHRONIC LIVER DISEASE ALCOHOL            & $63,405$ \\
CKDSTAGE3TO5                           

In [45]:
static.connect()
static.cursor.execute(""" SELECT *
                                FROM
                                	measurement_25_Hydroxyvitamin_D2_level_92
                                 ;""")
print(static.cursor.fetchone())

DEBUG:root:Connected to SQLite database


<class 'int'>


## Check query execution time

In [26]:
measurements.cursor.execute(f"""SELECT DISTINCT
                                    PATIENT_ID
                                FROM
                                    {mtable}
                                WHERE 
                                    PRACTICE_ID='20005'
                                """)
print(measurements.cursor.fetchall()[:100])

[(2960868120005,), (2960872920005,), (2960876820005,), (2960885820005,), (2960911920005,), (2960913820005,), (2960942420005,), (2960947320005,), (2960954920005,), (2960956720005,), (2960959120005,), (2960964320005,), (2960964620005,), (2981333420005,), (2984313520005,), (2984364120005,), (3142441820005,), (3231447320005,), (5410815220005,), (5610591420005,), (5610594420005,), (5690040620005,), (5690040920005,), (5702914820005,), (6060129520005,), (627688820005,), (628222420005,), (628591420005,), (628681820005,), (628841020005,), (629002720005,), (629950220005,), (629970320005,), (630276120005,), (630496820005,), (630637220005,), (630754220005,), (630825320005,), (630971220005,), (6606814520005,), (6618757120005,), (6621001220005,), (6688129820005,), (6717503120005,), (6910109220005,), (6917787120005,), (768817020005,), (8958004320005,), (8990528620005,), (8990529020005,), (8991433320005,), (8991433420005,)]


In [31]:
import time
start_time = time.time()
for mtable in measurements.measurement_table_names:
    measurements.cursor.execute(f"""SELECT
                                        *
                                    FROM
                                    	{mtable}
                                     WHERE
                                         PRACTICE_ID='20005' AND PATIENT_ID='30637220005'
                                    """)
    a = measurements.cursor.fetchall()
    # if len(a) > 0:
    #     print(a)

for table in ["diagnosis_table", "static_table"]:
    measurements.cursor.execute(f"""SELECT
                                        *
                                    FROM
                                    	{table}
                                     WHERE
                                         PRACTICE_ID='20005' AND PATIENT_ID='30637220005'
                                    """)
    
    # WHERE PRACTICE_PATIENT_ID = 'p20684_2602314020684' 
    a = measurements.cursor.fetchall()
    # print(a)
print(time.time()-start_time)

0.3020632266998291


# Collector demo

We can extract unique distinct column values from any table, and use these to chunk the data for processing

* We can add practice level conditions here,
    * e.g. practices that are in the North West region. 

In [9]:
from CPRD.data.dataset.collector import SQLiteDataCollector
import polars as pl
import logging 
logging.basicConfig(level=logging.INFO)

collector = SQLiteDataCollector(PATH_TO_DB)
collector.connect()

practice_ids = collector._extract_distinct(["static_table"], "PRACTICE_ID", conditions=["HEALTH_AUTH = 'North West'"])

print(f"The first ten unique practice IDs: {practice_ids[:10]} from {len(practice_ids)} total")

DEBUG:root:Connected to SQLite database
DEBUG:root:Query: SELECT DISTINCT PRACTICE_ID FROM static_table WHERE HEALTH_AUTH = 'North West'


The first ten unique practice IDs: [21505, 20995, 20996, 21508, 21000, 21514, 21515, 21007, 21008, 21010] from 305 total


## Given these practice IDs, we can chain this query to get the pratice patient ID's in a practice

* We do this as only the static table has all practice_patient_id, patient_id, and practice_id stored and indexed.
    * This means we can very quickly find the keys needed to index on the other tables which only have practice_patient_id indexed.
* We can add patient level conditions here
    * For example, only male, people who died during a period, etc

In [11]:
practice_patient_ids = []
for p_id in practice_ids:
    practice_patient_ids.append(collector._extract_distinct(["static_table"], "PATIENT_ID", conditions=[f"PRACTICE_ID = '{p_id}' AND SEX = 'M'"]))
                                
print(f"\nThe first five unique patient IDs from practice {practice_ids[0]}:\n\t{practice_patient_ids[0][:5]} from {len(practice_patient_ids[0])} total")
print(f"\nTotal male patients in each practice:\n\t {[len(ppid) for ppid in practice_patient_ids]}")

DEBUG:root:Query: SELECT DISTINCT PATIENT_ID FROM static_table WHERE PRACTICE_ID = '21505' AND SEX = 'M'
DEBUG:root:Query: SELECT DISTINCT PATIENT_ID FROM static_table WHERE PRACTICE_ID = '20995' AND SEX = 'M'
DEBUG:root:Query: SELECT DISTINCT PATIENT_ID FROM static_table WHERE PRACTICE_ID = '20996' AND SEX = 'M'
DEBUG:root:Query: SELECT DISTINCT PATIENT_ID FROM static_table WHERE PRACTICE_ID = '21508' AND SEX = 'M'
DEBUG:root:Query: SELECT DISTINCT PATIENT_ID FROM static_table WHERE PRACTICE_ID = '21000' AND SEX = 'M'
DEBUG:root:Query: SELECT DISTINCT PATIENT_ID FROM static_table WHERE PRACTICE_ID = '21514' AND SEX = 'M'
DEBUG:root:Query: SELECT DISTINCT PATIENT_ID FROM static_table WHERE PRACTICE_ID = '21515' AND SEX = 'M'
DEBUG:root:Query: SELECT DISTINCT PATIENT_ID FROM static_table WHERE PRACTICE_ID = '21007' AND SEX = 'M'
DEBUG:root:Query: SELECT DISTINCT PATIENT_ID FROM static_table WHERE PRACTICE_ID = '21008' AND SEX = 'M'
DEBUG:root:Query: SELECT DISTINCT PATIENT_ID FROM stati


The first five unique practice_patient IDs from practice 21505:
	[6499299221505, 6499427221505, 6499478421505, 6499452821505, 6499606421505] from 2978 total

Total male patients in each practice:
	 [2978, 7353, 167, 10075, 3456, 10727, 5948, 10060, 3337, 4355, 4359, 4732, 9742, 9991, 7067, 7739, 5589, 2995, 12730, 15698, 8748, 6097, 4769, 7536, 6467, 10999, 8421, 3352, 4423, 13347, 6035, 8694, 11087, 12412, 7594, 2431, 7837, 7384, 4038, 9033, 7273, 6921, 14112, 7506, 6299, 8669, 4480, 2326, 8275, 10382, 3230, 6515, 5382, 3210, 4623, 11147, 4061, 7983, 4202, 9844, 7802, 9414, 6508, 6570, 17641, 14467, 2733, 5349, 36498, 36236, 4853, 13566, 4878, 6545, 231, 6101, 5236, 7443, 7149, 5396, 5926, 6347, 9721, 19320, 3748, 9388, 7912, 10456, 4048, 12342, 12121, 6510, 12210, 11697, 15807, 7896, 10871, 7275, 2947, 8767, 11560, 9993, 7809, 22261, 7067, 4748, 3451, 6236, 9718, 12605, 1077, 5160, 6008, 5280, 4139, 6202, 16834, 3648, 4768, 68067, 11073, 10346, 6716, 12517, 2112, 6095, 3990, 2316, 4

### Create a generator which chunks the .db tables by practice_id or practice_patient_id and lazily batch using Polars (Rust)

* We can also choose to pass this method a list of conditions, one for each table.
   * For example, we may only want to collect measurements within some predfined list, period etc

In the first case, we may want to generate by individual level values. 

Here, we then pass in the list we generate over - which in this instance is the list of practice patient ID's belonging to the first chunk. Here we are then yielding one person per call to the generator.

In [13]:
generator = collector._lazy_generate_by_distinct(distinct_values=practice_ids, 
                                                 identifier_column="PRACTICE_ID",)
start = time.time()
for _idx, (chunk_name, lazy_table_frames_dict) in enumerate(generator):
    
    print(f"collector took {time.time() - start} seconds")

    start = time.time()
    batch = collector._collate_lazy_tables(lazy_table_frames_dict).collect()
    print(f"collating events took {time.time() - start} seconds")

    start = time.time()
    if _idx > 5:
        break
    
print(lazy_table_frames_dict)
# print(f"Chunk containing practice ID: {chunk_name}\n\n")
for key in lazy_table_frames_dict.keys():
    print(key)
    display(lazy_table_frames_dict[key].collect().head())


DEBUG:root:Query: SELECT * FROM static_table WHERE PRACTICE_ID = '21505'
DEBUG:root:loaded 1 practices from table in 0.08716821670532227 seconds/practice
DEBUG:root:Query: SELECT * FROM diagnosis_table WHERE PRACTICE_ID = '21505'
DEBUG:root:loaded 1 practices from table in 0.28239917755126953 seconds/practice
DEBUG:root:Query: SELECT * FROM measurement_25_Hydroxyvitamin_D2_level_92 WHERE PRACTICE_ID = '21505'
DEBUG:root:loaded 1 practices from table in 0.19316744804382324 seconds/practice
DEBUG:root:Query: SELECT * FROM measurement_25_Hydroxyvitamin_D3_level_90 WHERE PRACTICE_ID = '21505'
DEBUG:root:loaded 1 practices from table in 0.07250165939331055 seconds/practice
DEBUG:root:Query: SELECT * FROM measurement_AST___aspartate_transam_SGOT__46 WHERE PRACTICE_ID = '21505'
DEBUG:root:loaded 1 practices from table in 0.16443133354187012 seconds/practice
DEBUG:root:Query: SELECT * FROM measurement_AST_serum_level_47 WHERE PRACTICE_ID = '21505'
DEBUG:root:loaded 1 practices from table in 0.

collector took 18.426275491714478 seconds
collating events took 0.19113659858703613 seconds


DEBUG:root:loaded 1 practices from table in 0.12081551551818848 seconds/practice
DEBUG:root:Query: SELECT * FROM diagnosis_table WHERE PRACTICE_ID = '20995'
DEBUG:root:loaded 1 practices from table in 0.45955848693847656 seconds/practice
DEBUG:root:Query: SELECT * FROM measurement_25_Hydroxyvitamin_D2_level_92 WHERE PRACTICE_ID = '20995'
DEBUG:root:loaded 1 practices from table in 0.0019221305847167969 seconds/practice
DEBUG:root:Query: SELECT * FROM measurement_25_Hydroxyvitamin_D3_level_90 WHERE PRACTICE_ID = '20995'
DEBUG:root:loaded 1 practices from table in 0.006295919418334961 seconds/practice
DEBUG:root:Query: SELECT * FROM measurement_AST___aspartate_transam_SGOT__46 WHERE PRACTICE_ID = '20995'
DEBUG:root:loaded 1 practices from table in 0.10662031173706055 seconds/practice
DEBUG:root:Query: SELECT * FROM measurement_AST_serum_level_47 WHERE PRACTICE_ID = '20995'
DEBUG:root:loaded 1 practices from table in 0.1645832061767578 seconds/practice
DEBUG:root:Query: SELECT * FROM meas

collector took 15.557485580444336 seconds


DEBUG:root:Query: SELECT * FROM static_table WHERE PRACTICE_ID = '20996'
DEBUG:root:loaded 1 practices from table in 0.16534662246704102 seconds/practice
DEBUG:root:Query: SELECT * FROM diagnosis_table WHERE PRACTICE_ID = '20996'
DEBUG:root:loaded 1 practices from table in 0.008449077606201172 seconds/practice
DEBUG:root:Query: SELECT * FROM measurement_25_Hydroxyvitamin_D2_level_92 WHERE PRACTICE_ID = '20996'
DEBUG:root:loaded 1 practices from table in 0.0018794536590576172 seconds/practice
DEBUG:root:Query: SELECT * FROM measurement_25_Hydroxyvitamin_D3_level_90 WHERE PRACTICE_ID = '20996'
DEBUG:root:loaded 1 practices from table in 0.00196075439453125 seconds/practice
DEBUG:root:Query: SELECT * FROM measurement_AST___aspartate_transam_SGOT__46 WHERE PRACTICE_ID = '20996'
DEBUG:root:loaded 1 practices from table in 0.006097078323364258 seconds/practice
DEBUG:root:Query: SELECT * FROM measurement_AST_serum_level_47 WHERE PRACTICE_ID = '20996'
DEBUG:root:loaded 1 practices from table i

...

PRACTICE_ID,PATIENT_ID,EVENT,VALUE,DATE
i64,i64,str,f64,str
21515,5954187021515,"""Urine_albumin_…",6.29,"""2021-12-08"""
21515,5954187521515,"""Urine_albumin_…",2.11,"""2011-07-15"""
21515,5954189921515,"""Urine_albumin_…",2.33,"""2009-04-16"""
21515,5954189921515,"""Urine_albumin_…",1.88,"""2010-04-13"""
21515,5954189921515,"""Urine_albumin_…",2.11,"""2011-09-16"""


lazy_measurement_Urine_microalbumin_creatinine_ratio_36


PRACTICE_ID,PATIENT_ID,EVENT,VALUE,DATE
i64,i64,str,f64,str
21515,5954318021515,"""Urine_microalb…",null,"""2004-12-29"""
21515,5955241221515,"""Urine_microalb…",4.25,"""2013-09-13"""
21515,5955287921515,"""Urine_microalb…",1.0,"""2000-12-11"""
21515,5955287921515,"""Urine_microalb…",2.9,"""2000-12-16"""
21515,5955899121515,"""Urine_microalb…",1.1,"""2014-06-17"""


lazy_measurement_eGFR_using_creatinine_CKD_EPI_per_1_73_square_metres_33


PRACTICE_ID,PATIENT_ID,EVENT,VALUE,DATE
i64,i64,str,f64,str
21515,10995640321515,"""eGFR_using_cre…",90.0,"""2022-01-13"""
21515,10995640321515,"""eGFR_using_cre…",null,"""2022-01-13"""
21515,5954184321515,"""eGFR_using_cre…",90.0,"""2020-10-30"""
21515,5954184321515,"""eGFR_using_cre…",90.0,"""2021-10-19"""
21515,5954184321515,"""eGFR_using_cre…",null,"""2021-10-19"""


In [15]:
display(lazy_table_frames_dict)

{'lazy_static_table': <polars.LazyFrame object at 0x7F653F62A170>}

Alternatively, we may which to generate groups/batches of individuals for vectorisation. In that case we can pass in a list of lists. We generate over the outer list, and each call yields all the inner list values.

For example, in the below we have an outer list of all practices, and an inner list of patients within the practice. Consequently, each call to generate yields all records of all patients within a practice.

In [ ]:
# generator = collector._lazy_generate_by_distinct(practice_ids, ["static_table", "measurement_table", "diagnosis_table"], "PRACTICE_PATIENT_ID")
# for _idx, (chunk_name, lazy_table_frames_dict) in enumerate(generator):
#     break
    
# # print(f"Chunk containing practice ID: {chunk_name}\n\n")
# display(lazy_table_frames_dict["lazy_static"].collect().head())
# display(lazy_table_frames_dict["lazy_measurement"].collect().head())
# display(lazy_table_frames_dict["lazy_diagnosis"].collect().head())


### Test collation of generated lazy frames

In [15]:
lazy_batch = collector._collate_lazy_tables(lazy_table_frames_dict)

DEBUG:root:Dropping missing measurements
DEBUG:root:Removing patients with no observed events


### View how tables are aligned

In [16]:
batch = lazy_batch.collect()
display(batch)

first_identifier = list(batch["PRACTICE_PATIENT_ID"].unique())[0]
display(first_identifier)
                        
row = batch.filter(pl.col("PRACTICE_PATIENT_ID") == first_identifier)
display(row)


measur = lazy_table_frames_dict["lazy_measurement"].collect()
row = measur.filter(pl.col("EVENT") == "Diastolic_blood_pressure_5")
display(row)

PRACTICE_ID,PATIENT_ID,VALUE,EVENT,DAYS_SINCE_BIRTH,ETHNICITY,YEAR_OF_BIRTH,SEX,COUNTRY,HEALTH_AUTH,INDEX_DATE,START_DATE,END_DATE
i64,i64,list[f64],list[str],list[i64],str,datetime[μs],str,str,str,datetime[μs],datetime[μs],datetime[μs]
21515,5954183621515,"[59.5, 5.88]","[""O_E___height_1"", ""O_E___weight_2""]","[49, 49]","""WHITE""",2018-06-15 00:00:00,"""M""","""E""","""North West""",2019-06-18 00:00:00,2019-06-18 00:00:00,2022-03-19 00:00:00
21515,5954184321515,"[15.9, 161.5, … 12.2]","[""Body_mass_index_3"", ""O_E___height_1"", … ""Serum_free_T4_level_75""]","[5062, 5062, … 11477]","""WHITE""",1990-07-15 00:00:00,"""F""","""E""","""North West""",2005-05-14 00:00:00,2005-05-14 00:00:00,2022-03-19 00:00:00
21515,5954184721515,[3.9],"[""O_E___weight_2""]",[9],"""WHITE""",2011-02-15 00:00:00,"""M""","""E""","""North West""",2012-03-22 00:00:00,2012-03-22 00:00:00,2013-11-16 00:00:00
21515,5954185221515,"[64.0, 110.0]","[""Diastolic_blood_pressure_5"", ""Systolic_blood_pressure_4""]","[5140, 5140]","""MISSING""",2003-07-15 00:00:00,"""M""","""E""","""North West""",2005-01-01 00:00:00,2005-01-01 00:00:00,2022-03-19 00:00:00
21515,5954185521515,"[29.6, 70.0, … 140.0]","[""Body_mass_index_3"", ""Diastolic_blood_pressure_5"", … ""Systolic_blood_pressure_4""]","[8174, 8174, … 9158]","""MISSING""",1979-07-15 00:00:00,"""F""","""E""","""North West""",2005-01-01 00:00:00,2005-01-01 00:00:00,2005-01-20 00:00:00
21515,5954186221515,"[null, 110.0, 17.0]","[""ASTHMA_PUSHASTHMA"", ""O_E___height_1"", ""O_E___weight_2""]","[1481, 2716, 2716]","""MISSING""",1994-07-15 00:00:00,"""M""","""E""","""North West""",2005-01-01 00:00:00,2005-01-01 00:00:00,2008-02-07 00:00:00
21515,5954186421515,"[null, null]","[""ATOPICECZEMA"", ""ASTHMA_PUSHASTHMA""]","[392, 3782]","""WHITE""",2011-03-15 00:00:00,"""M""","""E""","""North West""",2012-04-01 00:00:00,2012-04-01 00:00:00,2022-03-19 00:00:00
21515,5954186521515,"[27.5, 60.0, … 11.3]","[""Body_mass_index_3"", ""Diastolic_blood_pressure_5"", … ""Serum_free_T4_level_75""]","[15797, 15797, … 20193]","""WHITE""",1965-07-15 00:00:00,"""F""","""E""","""North West""",2009-10-08 00:00:00,2009-10-08 00:00:00,2022-03-19 00:00:00
21515,5954186721515,"[17.21, 139.5, … 96.0]","[""Body_mass_index_3"", ""O_E___height_1"", … ""Systolic_blood_pressure_4""]","[4331, 4331, … 7514]","""MISSING""",2001-07-15 00:00:00,"""F""","""E""","""North West""",2005-01-01 00:00:00,2005-01-01 00:00:00,2022-03-19 00:00:00


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_2217528/1443966763.py:4 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2217528/1443966763.py'                      │
│                                                                                                  │
│ /rds/bear-apps/2022a/EL8-ice/software/polars/0.17.12-foss-2022a/lib/python3.10/site-packages/pol │
│ ars/dataframe/frame.py:1612 in __getitem__                                                       │
│                                                                                                  │
│   1609 │   │   # select single column                                                            │
│   1610 │   │   # df["foo"]                                                                       │
│   1611 │   │   if isinstance(item, str):                                                         │
│ ❱ 1612 │   │   │   return wrap_s(self._df.column(item))                                          │
│   1613 │   │                                                                                     │
│   1614 │   │   # df[idx]                                                                         │
│   1615 │   │   if isinstance(item, int):                                                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ColumnNotFoundError: PRACTICE_PATIENT_ID

# Demo wrapper

All of this is wrapped together (without the option to filter - add your own wrapper specific to you application)

In [ ]:
# for _idx, (chunk_name, lazy_table_frames_dict) in enumerate(collector.generator()):
#     lazy_batch = collector._collate_lazy_tables(**lazy_table_frames_dict)
#     break
# print(_idx)
# print(lazy_batch.collect())

In [ ]:
!jupyter nbconvert --to html --no-input buildV2.ipynb